In [2]:
import pandas as pd
import os
import openpyxl
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.styles import Border, Side, Alignment, Font, PatternFill
import subprocess

# === 1. Lê o arquivo TXT e converte em DataFrame ===
file_txt = "mapa2.TXT"

colunas = ["Lote", "N Pedido", "Produto", "Cor Etiq", "QTD Vol", "Data Corte", "Data Emb", "Status", "N/A"]
df = pd.read_csv(file_txt, skiprows=23, delimiter=";", names=colunas)
df = df.dropna(subset=["N Pedido"])
df = df.drop(["QTD Vol", "Data Corte", "Status", "N/A"], axis=1)

# === 2. Adiciona colunas manuais padrão ===
df["Isopor"] = "Não Conferido"
df["Tecnoplac"] = "Não Conferido"
df["Lote_C2"] = "Não Conferido"
df["Vidro"] = "Não Conferido"

# === 3. Merge com Mapa_Anterior.xlsx (se existir) ===
arquivo_anterior = "C:\git\linea\LineaBrasil\Isopor\Mapa_Inicio.xlsx"

if os.path.exists(arquivo_anterior):
    df_antigo = pd.read_excel(arquivo_anterior)

    # Faz o merge pelas colunas principais
    df = pd.merge(df, df_antigo[["Lote", "N Pedido", "Isopor", "Tecnoplac", "Lote_C2", "Vidro"]],
                  on=["Lote", "N Pedido"],
                  how="left",
                  suffixes=("", "_antigo"))

    for coluna in ["Isopor", "Tecnoplac", "Lote_C2", "Vidro"]:
        df[coluna] = df[f"{coluna}_antigo"].combine_first(df[coluna])
        df.drop(columns=[f"{coluna}_antigo"], inplace=True)

# === 4. Salva como Excel formatado ===
saida_excel = "Mapa_Producao.xlsx"
df.to_excel(saida_excel, index=False, engine='openpyxl')

# === 5. Abre o arquivo e aplica formatações com openpyxl ===   
wb = openpyxl.load_workbook(saida_excel)
sheet = wb.active

sheet.title = "Visualização"

# Validação de dados (listas suspensas)
status_isopor = '"Mapeado, Parcial, Entregue"'
rule_isopor = DataValidation(type='list', formula1=status_isopor, allow_blank=True)
rule_isopor.error = "Entrada Inválida"
rule_isopor.errorTitle = "Selecione Corretamente"
rule_isopor.prompt = "Selecione a Lista"
rule_isopor.promptTitle = "Selecione a opção"
sheet.add_data_validation(rule_isopor)
rule_isopor.add("F2:F250")

status_tecnoplac = '"Conferido, Não Conferido"'
rule_tecnoplac = DataValidation(type='list', formula1=status_tecnoplac, allow_blank=True)
rule_tecnoplac.error = "Entrada Inválida"
rule_tecnoplac.errorTitle = "Selecione Corretamente"
rule_tecnoplac.prompt = "Selecione a Lista"
rule_tecnoplac.promptTitle = "Selecione a opção"
sheet.add_data_validation(rule_tecnoplac)
rule_tecnoplac.add("G2:G250")

status_lote = '"Parcial, Separado, Entregue"'
rule_lote = DataValidation(type='list', formula1=status_lote, allow_blank=True)
rule_lote.error = "Entrada Inválida"
rule_lote.errorTitle = "Selecione Corretamente"
rule_lote.prompt = "Selecione a Lista"
rule_lote.promptTitle = "Selecione a opção"
sheet.add_data_validation(rule_lote)
rule_lote.add("H2:H250")

status_vidro = '"Separado, Entregue"'
rule_vidro = DataValidation(type='list', formula1=status_vidro, allow_blank=True)
rule_vidro.error = "Entrada Inválida"
rule_vidro.errorTitle = "Selecione Corretamente"
rule_vidro.prompt = "Selecione a Lista"
rule_vidro.promptTitle = "Selecione a opção"
sheet.add_data_validation(rule_vidro)
rule_vidro.add("I2:I250")


# Bordas finas
borda = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
    for cell in row:
        cell.border = borda

# Tamanho das colunas
larguras = {'A': 9, 'B': 11, 'C': 60, 'D': 11, 'E': 20, 'F': 15, 'G': 15, 'H': 15,'I': 15}
for col, largura in larguras.items():
    sheet.column_dimensions[col].width = largura

# Centraliza células (exceto Produto)
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
    for col_idx in [0, 1, 3, 4, 5, 6, 7, 8]:  # Produto (índice 2) fica alinhado à esquerda
        row[col_idx].alignment = Alignment(horizontal='center')

# Fonte padrão
fonte_padrao = Font(size=10)
for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
    for cell in row:
        cell.font = fonte_padrao

# Cabeçalho formatado
fonte_cabecalho = Font(size=11, bold=True, color="FFFFFF")
fundo_roxo = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
for cell in sheet["A1:I1"][0]:
    cell.font = fonte_cabecalho
    cell.fill = fundo_roxo
    cell.alignment = Alignment(horizontal="center", vertical="center")

# Cores alternadas nas linhas
cinza_escuro = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")
cinza_claro = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
for i, row in enumerate(sheet.iter_rows(min_row=2, max_row=sheet.max_row), start=0):
    fill = cinza_escuro if i % 2 == 0 else cinza_claro
    for cell in row:
        cell.fill = fill

sheet.freeze_panes = "A2"
wb.save(saida_excel)

# === 6. Abre o Excel ===
subprocess.Popen(['start', 'excel.exe', saida_excel], shell=True)




<>:23: SyntaxWarning: invalid escape sequence '\g'
<>:23: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Enzo\AppData\Local\Temp\ipykernel_17428\180631543.py:23: SyntaxWarning: invalid escape sequence '\g'
  arquivo_anterior = "C:\git\linea\LineaBrasil\Isopor\Mapa_Inicio.xlsx"


<Popen: returncode: None args: ['start', 'excel.exe', 'Mapa_Producao.xlsx']>